<a href="https://colab.research.google.com/github/Extralit/papers-ocr-benchmarks/blob/main/marker_demo_blocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Marker Demo: Research Paper Block Extraction & Exploration

Self-contained, Google Colab-ready.

- Upload a PDF, run Marker to extract JSON structure
- Flatten and filter block structure (inspired by TypeScript)
- Explore block types, metadata, tables, and figures
- No LLM-based summarization or captioning included


In [1]:
# 1. Install marker-pdf and dependencies
!uv pip install --quiet marker-pdf[full]
!uv pip install -q "PyMuPDF>=1.23.0" "pandas>=1.5.0"
!uv pip install -q "matplotlib>=3.5.0" "seaborn>=0.11.0" "textdistance>=4.6.0"
!uv pip install -q torch torchvision torchaudio

In [23]:
# prompt: import HTML and display from ipython

from IPython.display import HTML, display, JSON
from pprint import pprint

In [11]:
# 2. Upload a PDF
from google.colab import files
uploaded = files.upload()
pdf_path = next(iter(uploaded))

Saving Allossogbe_et_al_2017_Mal_J.pdf to Allossogbe_et_al_2017_Mal_J.pdf


In [12]:
file_path = "/content/Allossogbe_et_al_2017_Mal_J.pdf"

In [14]:
artifact_dict = create_model_dict()
artifact_dict

{'layout_model': <surya.layout.LayoutPredictor at 0x784a9e912010>,
 'recognition_model': <surya.recognition.RecognitionPredictor at 0x784aea2e9bd0>,
 'table_rec_model': <surya.table_rec.TableRecPredictor at 0x784ae0a19190>,
 'detection_model': <surya.detection.DetectionPredictor at 0x784af3725210>,
 'ocr_error_model': <surya.ocr_error.OCRErrorPredictor at 0x784ae9bf05d0>}

In [13]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.schema import BlockTypes

converter = PdfConverter(
    artifact_dict=create_model_dict(),
)

document = converter.build_document(file_path)
forms = document.contained_blocks((BlockTypes.Form,))

Running OCR Error Detection: 100%|██████████| 1/1 [00:00<00:00, 26.08it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 1/1 [00:01<00:00,  1.26s/it]


In [31]:
blocks = document.contained_blocks((BlockTypes.Text,))
blocks[0]

Text(polygon=PolygonBox(polygon=[[56.45703125, 229.614013671875], [506.076171875, 229.614013671875], [506.076171875, 257.422607421875], [56.45703125, 257.422607421875]], bbox=[56.45703125, 229.614013671875, 506.076171875, 257.422607421875]), block_description='A paragraph or line of text.', block_type=<BlockTypes.Text: '23'>, block_id=6, page_id=0, text_extraction_method='pdftext', structure=[/page/0/Line/40, /page/0/Line/58, /page/0/Line/64, /page/0/Line/67, /page/0/Line/77], ignore_for_output=False, replace_output_newlines=False, source='layout', top_k={<BlockTypes.Text: '23'>: 0.9404296875, <BlockTypes.TextInlineMath: '16'>: 0.05963134765625, <BlockTypes.Code: '10'>: 1.1980533599853516e-05, <BlockTypes.Footnote: '12'>: 4.947185516357422e-06, <BlockTypes.Form: '13'>: 4.827976226806641e-06}, metadata=None, lowres_image=None, highres_image=None, removed=False, has_continuation=False, blockquote=False, blockquote_level=0, html=None)

In [30]:
blocks[0].html

In [32]:
pprint(blocks)

[Text(polygon=PolygonBox(polygon=[[56.45703125, 229.614013671875], [506.076171875, 229.614013671875], [506.076171875, 257.422607421875], [56.45703125, 257.422607421875]], bbox=[56.45703125, 229.614013671875, 506.076171875, 257.422607421875]), block_description='A paragraph or line of text.', block_type=<BlockTypes.Text: '23'>, block_id=6, page_id=0, text_extraction_method='pdftext', structure=[/page/0/Line/40, /page/0/Line/58, /page/0/Line/64, /page/0/Line/67, /page/0/Line/77], ignore_for_output=False, replace_output_newlines=False, source='layout', top_k={<BlockTypes.Text: '23'>: 0.9404296875, <BlockTypes.TextInlineMath: '16'>: 0.05963134765625, <BlockTypes.Code: '10'>: 1.1980533599853516e-05, <BlockTypes.Footnote: '12'>: 4.947185516357422e-06, <BlockTypes.Form: '13'>: 4.827976226806641e-06}, metadata=None, lowres_image=None, highres_image=None, removed=False, has_continuation=False, blockquote=False, blockquote_level=0, html=None),
 Text(polygon=PolygonBox(polygon=[[62.27734375, 303.

In [ ]:
# 3. Run Marker to extract JSON structure
import os
output_dir = 'marker_output'
os.makedirs(output_dir, exist_ok=True)
json_out = os.path.join(output_dir, os.path.splitext(os.path.basename(pdf_path))[0] + '_structure.json')

!marker_single "{pdf_path}" --output_format json --output_dir "{output_dir}"

In [ ]:
# 4. Load the Marker JSON
import json
with open(json_out, 'r') as f:
    marker_json = json.load(f)

In [ ]:
# 5. Data models and flattening utilities (Python version of your TypeScript)
from typing import List, Dict, Any

class SimplifiedBlock:
    def __init__(self, type: str, content: str, page: int, bbox: list):
        self.type = type
        self.content = content
        self.page = page
        self.bbox = bbox

    def as_dict(self):
        return {
            'type': self.type,
            'content': self.content,
            'page': self.page,
            'bbox': self.bbox,
        }

import html

def decode_html_entities(text: str) -> str:
    return html.unescape(text)

def flatten_marker_json(blocks: List[Dict[str, Any]], page_number: int = 0) -> List[SimplifiedBlock]:
    flat_blocks = []
    for block in blocks:
        # Skip Page blocks but process their children
        if block.get('block_type') == 'Page':
            child_page = int(block.get('id', '0/0/0').split('/')[2]) if 'id' in block else 0
            flat_blocks.extend(flatten_marker_json(block.get('children', []), child_page))
            continue

        # Process current block
        content = ''
        if block.get('images') and isinstance(block['images'], dict) and block['images']:
            content = next(iter(block['images'].values()))
        elif block.get('block_type') == 'Table':
            content = block.get('html', '').strip()
        elif block.get('html'):
            import re
            content = re.sub(r'<[^>]*>', ' ', block['html']).strip()
        content = decode_html_entities(content)

        page = (int(block.get('id', '0/0/0').split('/')[2]) if 'id' in block else page_number) + 1
        bbox = block.get('bbox', [0,0,0,0])

        flat_blocks.append(SimplifiedBlock(
            type=block.get('block_type', ''),
            content=content,
            page=page,
            bbox=bbox
        ))

        # Recursively process children (except for Page blocks)
        if block.get('children'):
            flat_blocks.extend(flatten_marker_json(block['children'], page))
    return flat_blocks

def filter_and_flatten_marker_json(blocks: List[Dict[str, Any]], page_number: int = 0) -> List[SimplifiedBlock]:
    unfiltered = flatten_marker_json(blocks, page_number)
    remove_types = {
        'TableCell', 'TableGroup', 'FigureGroup', 'ListGroup', 'Reference',
        'PageFooter', 'PageHeader', 'Footnote'
    }
    return [b for b in unfiltered if b.type not in remove_types and b.content]

In [ ]:
# 6. Flatten and filter the Marker output
flat_blocks = filter_and_flatten_marker_json(marker_json.get('children', []))

In [ ]:
# 7. Explore block types and content
import pandas as pd

df = pd.DataFrame([b.as_dict() for b in flat_blocks])
print('Block types found:', df['type'].unique())
df.head(20)  # Show first 20 blocks

In [ ]:
# 8. Simple metadata extraction (title, authors, abstract)
def extract_metadata(blocks: List[SimplifiedBlock]):
    title = next((b.content for b in blocks if b.type.lower() in {'title', 'main_title'}), '')
    authors = next((b.content for b in blocks if 'author' in b.type.lower()), '')
    abstract = next((b.content for b in blocks if 'abstract' in b.type.lower()), '')
    return {'title': title, 'authors': authors, 'abstract': abstract}

metadata = extract_metadata(flat_blocks)
print('Extracted Metadata:', metadata)

In [ ]:
# 9. Find and display all tables and figures (with extensibility for custom processing)
tables = [b for b in flat_blocks if b.type == 'Table']
figures = [b for b in flat_blocks if b.type == 'Figure' or b.type == 'Picture']

print(f'Found {len(tables)} tables and {len(figures)} figures.')

# Example: Show first table's HTML (for further processing)
if tables:
    from IPython.display import display, HTML
    print('First table HTML:')
    display(HTML(tables[0].content))

# Example: Show first figure as image (if base64-encoded)
import base64
from IPython.display import Image

def show_base64_image(b64str):
    try:
        display(Image(data=base64.b64decode(b64str)))
    except Exception as e:
        print('Could not display image:', e)

if figures:
    print('First figure (if image):')
    show_base64_image(figures[0].content)

In [ ]:
# 10. (Optional) Extensible: Add your own logic to process tables/figures, e.g., send table HTML to a model, extract captions, etc.
# (No LLM-based summarization or captioning included)

In [ ]:
# 11. Save flattened blocks for further analysis
df.to_json('flattened_blocks.json', orient='records', indent=2)
from google.colab import files
files.download('flattened_blocks.json')